# Aggregations

**Aggregation** = Summarizing data by applying a function over groups or entire datasets

- Usually **group by one or more keys (columns)** and apply an **aggregation function** (like sum, count, avg).
- Used for **reporting, dashboards, statistics, KPIs**

### 6 Types of Aggregations in Spark

**1. Simple Aggregation:** Applies aggregation to the *entire DataFrame.*
    
    - Total items sold and average price across all transactions.

**2. groupBy:** Group rows by one or more columns and apply aggregations.
    
    - Show how many items were sold by country.

**3. Window Aggregation:** Similar to groupBY, but operates *relative to the current row* (eg. moving averages, rankings)
    
    - Rank sales records within each country.

**4. Grouping Sets:** Combine different groupings in one query (e.g., group by Country, and Product, and both)
    
    - Combine multiple level of aggregation in one query (e.g. total by country and total by product separately.

**5. Rollup (Heirarchial Summary:** Computes *sub-total levels* moving top-down in hierarchy.
    
    - Show subtotals by country, then a grand total at the bottom.

**6. Cube (All Combinations:** Like rollup, but also computes *cross-tabulated summaries*.
    
    - Show totals by country, totals by product, and grand total - all in one.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Aggregations") \
    .getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/18 19:42:13 WARN Utils: Your hostname, Satkars-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.101 instead (on interface en0)
25/07/18 19:42:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/18 19:42:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# lets begin by reading in our data on purchases + repartition + cache for rapid access:

df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Users/satkarkarki/spark_the_definitive_guide/data/retail-data/all/online-retail-dataset.csv") \
    .coalesce(5)

df.cache()
df.createOrReplaceTempView("dfTable")


In [3]:
df.createOrReplaceTempView("dfTable")

In [4]:
spark.sql("SELECT * FROM dfTable LIMIT 5").show()


+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+



In [5]:
df.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows


## Aggregation Functions

- All aggregations are available as functions.
- Most aggregation functions can be called from the **pyspark.sql.functions** in PySpark.

##### Count is actually an action opposed to a transformation, so it returns immediately.
    - Use count to get an idea of the total size of the dataset.

In [6]:
 ## Applying the most basic aggregation i.e. count to the entire DataFrame:

df.count()

541909

In [7]:
df.count() == 541909

True

#### Using count() as a function

In [8]:
from pyspark.sql.functions import count
df.select(count("StockCode")).show()

## In SQL:
## SELECT COUNT(*) FROM dfTable

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



##### countDisctinct() 

In [9]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct("StockCode")).show()

## In SQL:
## SELECT COUNT(DISTINCT *) FROM dfTable

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



##### approx_count_distinct
    - useful when working with large datasets
    - approximates the count till a certain degree of accuracy


In [10]:
from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("StockCode", 0.1)).show()

## IN SQL:
## SELECT approx_count_distinct(StockCode, 0.1 FROM dfTable

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



##### first and last
 - retrieves first and last values (obvious from the name)

In [11]:
from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()

## In SQL:
## SELECT first(StockCode), last(StockCode) FROM dfTable

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



##### min and max
  - Extract the minimum and maximum values from a DataFrame.

In [12]:
from pyspark.sql.functions import min, max
df.select(min("Quantity"), max("Quantity")).show(2) 

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



##### sum
    - Another simple task is to add all the values in a row using the sum function.

In [13]:
from pyspark.sql.functions import sum
df.select(sum("Quantity")).show()


+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [14]:
## Another variant is the sumDistinct function 
from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("Quantity")).show()


+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



/Users/satkarkarki/anaconda3/lib/python3.11/site-packages/pyspark/sql/functions/builtin.py:1780: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


##### avg
- the average function (sum/count)

In [15]:
from pyspark.sql.functions import sum, count, avg, expr

df.select(
        count("Quantity").alias("total_transactions"),
        sum("Quantity").alias("total_purchases"),
        avg("Quantity").alias("avg_purchases"),
        expr("mean(Quantity)").alias("mean_purchases")) \
    .selectExpr(
        "total_purchases/total_transactions",
        "avg_purchases",
        "mean_purchases").show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



##### Variance and Standard Deviation

In [16]:
from pyspark.sql.functions import var_pop, stddev_pop, var_samp, stddev_samp
df.select(var_pop("Quantity"), var_samp("Quantity"),
         stddev_pop("Quantity"), stddev_samp("Quantity")).show()

# -- in SQL
# SELECT var_pop(Quantity), var_samp(Quantity),
# stddev_pop(Quantity), stddev_samp(Quantity)
# FROM dfTable

+------------------+------------------+--------------------+---------------------+
| var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
|47559.303646609274| 47559.39140929897|  218.08095663447847|   218.08115785023466|
+------------------+------------------+--------------------+---------------------+



##### Skewness and Kurtosis

- lets calculate the measureemnts of extreme points the head and the tail around the mean

In [17]:
# in Python
from pyspark.sql.functions import skewness, kurtosis
df.select(skewness("Quantity"), kurtosis("Quantity")).show()


# -- in SQL
# SELECT skewness(Quantity), kurtosis(Quantity) FROM dfTable

+-------------------+------------------+
| skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
|-0.2640755761052443|119768.05495533928|
+-------------------+------------------+



##### covariance and correlation:

- functions used to compare the interactions of two values in two differenct columns together

In [18]:
from pyspark.sql.functions import corr, covar_pop, covar_samp
df.select(corr("Quantity", "Quantity"), covar_samp("Quantity", "Quantity"), 
          covar_pop("Quantity", "Quantity")).show()

+------------------------+------------------------------+-----------------------------+
|corr(Quantity, Quantity)|covar_samp(Quantity, Quantity)|covar_pop(Quantity, Quantity)|
+------------------------+------------------------------+-----------------------------+
|                     1.0|             47559.39140929898|            47559.30364660928|
+------------------------+------------------------------+-----------------------------+



##### Aggregating to Complex Types

- Summarizes a column by collecting all unique values (set) and all values incl. duplicates (list) into arrays.



In [19]:
from pyspark.sql.functions import collect_set, collect_list
df.agg(collect_set("Country"), collect_list("Country")).show()

## In SQL
## SELECT collect_set(Country), collect_set(Country) FROM dfTable

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Netherlands, Sau...| [United Kingdom, ...|
+--------------------+---------------------+



### Grouping

- A more common task is to perform calculations based on **groups** in the data.
- Grouping is typically done on categorical data.
- Let's kick things off with some simple grouping.

In [20]:
df.groupBy("InvoiceNo", "CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
|   538800|     16458|   10|
|   538942|     17346|   12|
|  C539947|     13854|    1|
|   540096|     13253|   16|
|   540530|     14755|   27|
|   541225|     14099|   19|
|   541978|     13551|    4|
|   542093|     17677|   16|
|   543188|     12567|   63|
|   543590|     17377|   19|
|  C543757|     13115|    1|
|  C544318|     12989|    1|
|   544578|     12365|    1|
|   545165|     16339|   20|
|   545289|     14732|   30|
+---------+----------+-----+
only showing top 20 rows


###### Grouping with Expressions

- Group data and apply aggregations using both function-style and SQL-expression style

In [21]:
from pyspark.sql.functions import count

df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)")).show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
|   537252|   1|              1|
|   537691|  20|             20|
|   538041|   1|              1|
|   538184|  26|             26|
|   538517|  53|             53|
|   538879|  19|             19|
|   539275|   6|              6|
|   539630|  12|             12|
|   540499|  24|             24|
|   540540|  22|             22|
|  C540850|   1|              1|
|   540976|  48|             48|
|   541432|   4|              4|
|   541518| 101|            101|
|   541783|  35|             35|
|   542026|   9|              9|
|   542375|   6|              6|
|  C542604|   8|              8|
+---------+----+---------------+
only showing top 20 rows


#### Grouping with Maps

- Groups data and applies multiple SQL-style aggregations in-line using expressions.

In [22]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)"))\
    .show()

## IN SQL:
## SELECT avg(Quantity), stddev_pop(Quantity), InvoiceNo FROM dfTable
## GROUP BY InvoiceNo

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1

## Window Functions

- A `GROUP BY` aggregates data so that each group becomes a single row.

**Example:** Grouping by `ClientID` gives you one row per client, such as their total lifetime spend.

- A **Window Function** lets you compute a value for every row, while still referencing a group of rows (called a *window* or *frame*).

**Example:** For each transaction row, you can compute that client’s total spend over 5 years or their running total to date, without collapsing the rows.


In [23]:
## Let's convert the invoice date to contain only date info (not time info)

from pyspark.sql.functions import col, to_date
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "M/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

###### The first step to a window function is to create a window specification
- **Note:** `partitionBy` is unrelated to the partionining schem concept covered thus far
- `rowsBetween` defines the **frame** for the window function, commonly used for **cumulative aggregations**.

In [24]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc
windowSpec = Window \
    .partitionBy("CustomerId", "date") \
    .orderBy(desc("Quantity")) \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [25]:
## Applying an aggregation function to learn the maximum purchase quantity over all time:
## This code will return a column which we can use in SELECT statement later:

from pyspark.sql.functions import max
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

In [26]:
## Before proceeding to use the SELECT statement, we will create the purchase quantity rank (use dense_rank)
## This code will return a column which we can use in SELECT statement later:

from pyspark.sql.functions import dense_rank, rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

In [27]:
## Using SELECT statement to view the calculated window values:

from pyspark.sql.functions import col

dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId") \
    .select(
        col("CustomerId"),
        col("date"),
        col("Quantity"),
        purchaseRank.alias("quantityRank"),
        purchaseDenseRank.alias("quantityDenseRank"),
        maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

## Grouping Sets

- `Grouping Sets` lets you calculate **multiple** `GROUP BY` **aggregations** in a **single query**.
- We'll have to use **Spark SQL** to perform grouping sets.



In [28]:
# simple data cleaning aka null-cleansing:

dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")

In [29]:
spark.sql("""
    SELECT 
        CustomerId, 
        stockCode, 
        SUM(Quantity) AS totalQuantity
    FROM dfNoNull
    GROUP BY GROUPING SETS (
        (CustomerId, stockCode),
        (CustomerId),
        (stockCode),
        ()
    )
    ORDER BY CustomerId DESC NULLS LAST, stockCode DESC NULLS LAST
""").show()


+----------+---------+-------------+
|CustomerId|stockCode|totalQuantity|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
|     18287|    47422|           24|
|     18287|    47421|           48|
|     18287|    35967|           36|
|     18287|    23445|           20|
|     18287|    23378|           24|
|     18287|    23376|           48|
|     18287|    23310|           36|
|     18287|    23274|           12|
|     18287|    23272|           12|
|     18287|    23269|           36|
+----------+---------+-------------+
only showing top 20 rows


#### ROLLUP

- `ROLLUP` is a shortcut in Spark that automatically performs multiple levels of aggregation in **hierarchical order.**

In [30]:
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity")) \
    .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity") \
    .orderBy("Date")
rolledUpDF.show()

+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      NULL|          NULL|       5176450|
|2010-12-01|United Kingdom|         23949|
|2010-12-01|        France|           449|
|2010-12-01|     Australia|           107|
|2010-12-01|          NULL|         26814|
|2010-12-01|        Norway|          1852|
|2010-12-01|       Germany|           117|
|2010-12-01|          EIRE|           243|
|2010-12-01|   Netherlands|            97|
|2010-12-02|          EIRE|             4|
|2010-12-02|          NULL|         21023|
|2010-12-02|United Kingdom|         20873|
|2010-12-02|       Germany|           146|
|2010-12-03|      Portugal|            65|
|2010-12-03|        Poland|           140|
|2010-12-03|       Belgium|           528|
|2010-12-03|United Kingdom|         10439|
|2010-12-03|        France|           239|
|2010-12-03|         Italy|           164|
|2010-12-03|          NULL|         14830|
+----------

In [31]:
rolledUpDF.where("Country IS NULL").show()
rolledUpDF.where("Date IS NULL").show()

+----------+-------+--------------+
|      Date|Country|total_quantity|
+----------+-------+--------------+
|      NULL|   NULL|       5176450|
|2010-12-01|   NULL|         26814|
|2010-12-02|   NULL|         21023|
|2010-12-03|   NULL|         14830|
|2010-12-05|   NULL|         16395|
|2010-12-06|   NULL|         21419|
|2010-12-07|   NULL|         24995|
|2010-12-08|   NULL|         22741|
|2010-12-09|   NULL|         18431|
|2010-12-10|   NULL|         20297|
|2010-12-12|   NULL|         10565|
|2010-12-13|   NULL|         17623|
|2010-12-14|   NULL|         20098|
|2010-12-15|   NULL|         18229|
|2010-12-16|   NULL|         29632|
|2010-12-17|   NULL|         16069|
|2010-12-19|   NULL|          3795|
|2010-12-20|   NULL|         14965|
|2010-12-21|   NULL|         15467|
|2010-12-22|   NULL|          3192|
+----------+-------+--------------+
only showing top 20 rows
+----+-------+--------------+
|Date|Country|total_quantity|
+----+-------+--------------+
|NULL|   NULL|       

##### Cube

- A cube takes the rollup to a level deeper.


In [ ]:
from pyspark.sql.functions import sum

dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))

##### Grouping Metadata

- Assigns a numeric code to each groups being aggregated so we know which row belongs to which level of aggregation.

## Pivot in Spark

- Turn rows into columns and vice-versa.

In [32]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

In [34]:
pivoted.where("date > '2011-12-05'").select("date" ,"`USA_sum(Quantity)`").show()

+----------+-----------------+
|      date|USA_sum(Quantity)|
+----------+-----------------+
|2011-12-06|             NULL|
|2011-12-09|             NULL|
|2011-12-08|             -196|
|2011-12-07|             NULL|
+----------+-----------------+

